# 문제 정의

제공 받은 Dataset은 age, anaemia, high blood pressure, diabetes 등의 병원데이터라고 추측되는 데이터이고 DEATH_EVENT는 해당 환자의 사후관리 기간 중 사망한 경우를 나타냅니다.

라이브러리 임포트

In [ ]:
# Libraies for reading and handling the data
import numpy as np
import pandas as pd
import re

# Libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Libraries for creating ML model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Library for Analysing the ML model
from sklearn import metrics

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


데이터 준비하기

In [ ]:
# csv 파일을 데이터프레임으로 변환
df = pd.read_csv('data.csv', header=None)

In [ ]:
# 열 이름 지정
df.columns = ['age','anamia','creatinine_phosphokinase','diabetes','ejection_fraction',
              'high_blood_pressure','platelets','serum_creatinine','serum_sodium',
              'sex','smoking','time','DEATH_EVENT']

데이터 탐색 및 데이터 전처리

In [ ]:
# column 추가로 중복된 첫 번째행 삭제
df= df.drop(0,0)
df.head()

,age,anamia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000,2.7,116,0,0,8,1


In [ ]:
# age 열의 자료형 변경 (문자열 -> 숫자)
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', '60.667', '73', '77',
       '78', '54', '47', '56'], dtype=object)

In [ ]:
# 60.667를 np.nan으로 변경
df['age'].replace('60.667',np.nan,inplace=True)
df.info

<bound method DataFrame.info of     age anamia creatinine_phosphokinase diabetes  ... sex smoking time DEATH_EVENT
1    75      0                      582        0  ...   1       0    4           1
2    55      0                     7861        0  ...   1       0    6           1
3    65      0                      146        0  ...   1       1    7           1
4    50      1                      111        0  ...   1       0    7           1
5    65      1                      160        1  ...   0       0    8           1
..   ..    ...                      ...      ...  ...  ..     ...  ...         ...
295  62      0                       61        1  ...   1       1  270           0
296  55      0                     1820        0  ...   0       0  271           0
297  45      0                     2060        1  ...   0       0  278           0
298  45      0                     2413        0  ...   1       1  280           0
299  50      0                      196        0  ...  

In [ ]:
# nan 확인
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', nan, '73', '77', '78',
       '54', '47', '56'], dtype=object)

In [ ]:
# 데이터안 nan값 60으로 변경
df=df.fillna(60)

In [ ]:
# 변경 사항 확인
df['age'].unique()

array(['75', '55', '65', '50', '90', '60', '80', '62', '45', '49', '82',
       '87', '70', '48', '68', '53', '95', '58', '94', '85', '69', '72',
       '51', '57', '42', '41', '67', '79', '59', '44', '63', '86', '66',
       '43', '46', '61', '81', '52', '64', '40', 60, '73', '77', '78',
       '54', '47', '56'], dtype=object)

In [ ]:
# 열들의 자료형 변경 (문자열 -> 숫자)
df['age']=df['age'].astype('int')

df['creatinine_phosphokinase']=df['creatinine_phosphokinase'].astype('int')

df['ejection_fraction']=df['ejection_fraction'].astype('int')

df['platelets']=df['platelets'].astype('float')

df['serum_creatinine']=df['serum_creatinine'].astype('float')

df['serum_sodium']=df['serum_sodium'].astype('int')

df['time']=df['time'].astype('int')

df['DEATH_EVENT']=df['DEATH_EVENT'].astype('int')

In [ ]:
# 숫자 형태의 변수 -> 카테고리 형태로 변경
categorical_var=['anamia','diabetes','high_blood_pressure','sex','smoking']
df[categorical_var] = df[categorical_var].astype('category')

형변환 완료한 데이터 형식

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 1 to 299
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   age                       299 non-null    int64   
 1   anamia                    299 non-null    category
 2   creatinine_phosphokinase  299 non-null    int64   
 3   diabetes                  299 non-null    category
 4   ejection_fraction         299 non-null    int64   
 5   high_blood_pressure       299 non-null    category
 6   platelets                 299 non-null    float64 
 7   serum_creatinine          299 non-null    float64 
 8   serum_sodium              299 non-null    int64   
 9   sex                       299 non-null    category
 10  smoking                   299 non-null    category
 11  time                      299 non-null    int64   
 12  DEATH_EVENT               299 non-null    int64   
dtypes: category(5), float64(2), int64(6)
memory usage:

In [ ]:
df.head()

,age,anamia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
1,75,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
2,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
3,65,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
4,50,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
5,65,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


데이터 분리하기

In [ ]:
# X(시험지), y(정답지) 분리하기
X = df.iloc[:,:-1]
y = df['DEATH_EVENT']

In [ ]:
# 원-핫 인코딩 처리
# 컬럼 -> 원-핫 인코딩 처리
temp = pd.get_dummies(X[categorical_var], drop_first=True)

In [ ]:
# 기존 데이터랑 합치기
X_modified = pd.concat([X,temp], axis=1)

In [ ]:
# 기존 컬럼 삭제
X_modified.drop(categorical_var, axis=1, inplace=True)

In [ ]:
X_modified.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anamia_1,diabetes_1,high_blood_pressure_1,sex_1,smoking_1
1,75,582,20,265000.00,1.9,130,4,0,0,1,1,0
2,55,7861,38,263358.03,1.1,136,6,0,0,0,1,0
3,65,146,20,162000.00,1.3,129,7,0,0,0,1,1
4,50,111,20,210000.00,1.9,137,7,1,0,0,1,0
5,65,160,20,327000.00,2.7,116,8,1,1,0,0,0


훈련용, 테스트 데이터 분리하기

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_modified,y,
                                                    train_size=0.7, # test_size
                                                    random_state=7)

NN 분류 모형 - sklearn 사용

In [ ]:
# sklearn 라이브러리에서 NN 분류 모형 가져오기
from sklearn.neural_network import MLPClassifier

# 훈련 세트 각 특성의 평균을 계산
mean_on_train = X_train.mean(axis=0)
# 훈련 세트 각 특성의 표준 편차를 계산
std_on_train = X_train.std(axis=0)

# 데이터에서 평균을 빼고 표준 편차로 나누면
# 평균 0, 표준 편차 1인 데이터로 변환
X_train_scaled = (X_train - mean_on_train) / std_on_train 
# (훈련 데이터의 평균과 표준 편차를 이용해) 같은 변환을 테스트 세트에도 합니다.
X_test_scaled = (X_test - mean_on_train) / std_on_train 

nn = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto',
                   beta_1=0.9, beta_2=0.999, early_stopping=False,
                   epsilon=1e-08, hidden_layer_sizes=(30, 30, 30),
                   learning_rate='constant', learning_rate_init=0.001,
                   max_iter=200, momentum=0.9, nesterovs_momentum=True,
                   power_t=0.5, random_state=7, shuffle=True, solver='adam',
                   tol=0.0001, validation_fraction=0.1, verbose=False,
                   warm_start=False)

In [ ]:
# 훈련 데이터를 사용하여 모형 학습
nn.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=7, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
# weights 확인
nn.coefs_

[array([[-3.45214697e-01,  2.26713296e-01, -6.87951824e-08,
          1.88599485e-01,  1.85041602e-01,  3.14927203e-02,
         -2.43210946e-02, -3.46070866e-01, -3.41857242e-02,
         -7.39329552e-03,  1.34221162e-02,  7.24636908e-02,
         -6.98814538e-02, -1.55174616e-01, -1.43561693e-01,
          3.29577659e-01, -2.37890950e-01, -5.82952503e-02,
          1.53256137e-01, -2.58813242e-01,  1.22205742e-02,
          1.66019763e-01, -2.25199937e-01,  2.97972921e-02,
          3.30307962e-01, -3.01021776e-01,  1.41924072e-08,
          4.33342763e-02,  1.45624038e-01,  1.44398547e-08],
        [-2.34853539e-01, -2.70713055e-03, -2.16618944e-03,
         -9.91278004e-03, -3.00072959e-03,  2.42558391e-01,
          1.89687187e-01, -1.52863355e-01, -1.54369855e-06,
         -1.58916925e-01,  8.36239409e-08, -5.15986903e-03,
          1.27903172e-01, -2.40983232e-03, -1.62244397e-02,
          1.73289285e-01, -7.57385206e-02,  2.97331793e-01,
         -8.17998416e-02,  6.80470762e-

In [ ]:
# layer shape 확인
[coef.shape for coef in  nn.coefs_]

[(12, 30), (30, 30), (30, 30), (30, 1)]

모델 성능 평가

In [ ]:
# NN 모델 성능 평가
from sklearn import metrics

print('테스트 세트 정확도: {:.2f}%'.format(nn.score(X_test, y_test)*100))

테스트 세트 정확도: 72.22%
